[View in Colaboratory](https://colab.research.google.com/github/heumchri/cityscapesScripts/blob/master/downloadCityscapes.ipynb)

# general

In [0]:
#!kill -9 -1

## utilization monitoring

In [0]:
#requirements for gpu and ram usage

# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize


In [0]:
#gpu and ram usage

import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " I Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

In [0]:
#running processes

#!ps -aux
!ps -aux | grep python

In [0]:
#storage usage

!df -h 

# download cityscapes dataset

In [0]:
cd /content/

In [0]:
!mkdir datasets

In [0]:
cd datasets/

In [0]:
!mkdir cityscapes

In [0]:
cd cityscapes/

In [0]:
!wget --keep-session-cookies --save-cookies=cookies.txt --post-data 'username=heumann.christopher&password=xxx&submit=Login' https://www.cityscapes-dataset.com/login/

In [0]:
!wget --load-cookies cookies.txt --content-disposition https://www.cityscapes-dataset.com/file-handling/?packageID=1
!wget --load-cookies cookies.txt --content-disposition https://www.cityscapes-dataset.com/file-handling/?packageID=3

In [0]:
!unzip gtFine_trainvaltest.zip

In [0]:
!rm README license.txt

In [0]:
!unzip leftImg8bit_trainvaltest.zip

In [0]:
!rm README cookies.txt index.html index.html.1 license.txt gtFine_trainvaltest.zip leftImg8bit_trainvaltest.zip


# install cityscapesscripts

In [0]:
!pip2 install cython

In [0]:
cd /content/

In [0]:
!git clone https://github.com/heumchri/cityscapesScripts.git

In [0]:
cd cityscapesScripts/

In [0]:
!python2 setup.py build_ext --inplace

In [0]:
import os
os.environ['CITYSCAPES_DATASET'] = '/content/datasets/cityscapes/'

## generate binary training images

In [0]:
!python2 /content/cityscapesScripts/cityscapesscripts/preparation/createTrainIdLabelImgs.py

## visualize results

In [0]:
from IPython.display import Image
#rgb image
Image('/content/datasets/cityscapes/leftImg8bit/train/aachen/aachen_000000_000019_leftImg8bit.png')

In [0]:
#colored ground truth
Image('/content/datasets/cityscapes/gtFine/train/aachen/aachen_000000_000019_gtFine_color.png')

In [0]:
#binary ground truth
Image('/content/datasets/cityscapes/gtFine/train/aachen/aachen_000000_000019_gtFine_labelTrainIds.png')